<a href="https://colab.research.google.com/github/michaelknorr/CHE4061/blob/main/CHE4061_Flash_Drum_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np

# Inputs
phi_0 = float(input("Input an initial guess for phi: "))
num_components = int(input("Input the number of streams feeding the flash drum: "))
comp_x = np.array(list(map(float, input("Input a list of molar flow rates for each stream (comma-separated): ").split(','))))
K = np.array(list(map(float, input("Input the K (equilibrium constant) values for each stream in order (comma-separated): ").split(','))))


Input an initial guess for phi: 0.5
Input the number of streams feeding the flash drum: 2
Input a list of molar flow rates for each stream (comma-separated): 30, 70
Input the K (equilibrium constant) values for each stream in order (comma-separated): 3.2, 0.47


In [31]:
import numpy as np

def Newton_Raphson(phi_0, num_components, comp_x, K, tol=1e-6, max_iter=100, show_table=True):
    """
    Performs Newton-Raphson iteration to find the root of a function.

    Parameters:
    phi_0 : float
        Initial guess for phi.
    num_components : int
        Number of components.
    comp_x : list or numpy array
        Mole fractions of components.
    K : list or numpy array
        Equilibrium constants of components.
    tol : float, optional
        Convergence tolerance (default is 1e-6).
    max_iter : int, optional
        Maximum number of iterations (default is 100).
    show_table : bool, optional
        Whether to print the iteration table (default is True).

    Returns:
    phi_next : float
        Converged phi value.
    """
    phi_current = phi_0

    if show_table:
        print(f"{'Iteration':<10}{'Phi':<15}{'f(Phi)':<20}{'f(Phi)':<20}{'Phi_next':<15}{'Error':<15}")
        print("-" * 95)

    for iteration in range(max_iter):
        f_phi_current = np.sum([(comp_x[i] * (1 - K[i])) / (1 + phi_current * (K[i] - 1)) for i in range(num_components)])
        f_phi_next = np.sum([(comp_x[i] * (1 - K[i])**2) / (1 + phi_current * (K[i] - 1))**2 for i in range(num_components)])

        phi_next = phi_current - f_phi_current / f_phi_next
        error = abs(phi_next - phi_current)

        if show_table:
            print(f"{iteration:<10}{phi_current:<15.6f}{f_phi_current:<20.6f}{f_phi_next:<20.6f}{phi_next:<15.6f}{error:<15.6f}")

        if error < tol:
            return phi_next  # Converged solution

        phi_current = phi_next

    raise ValueError("Newton-Raphson did not converge within the maximum number of iterations")


In [36]:
Newton_Raphson(phi_0, num_components, comp_x, K) #GT3 Problem 1 Part A Work

Iteration Phi            f(Phi)              f(Phi)              Phi_next       Error          
-----------------------------------------------------------------------------------------------
0         0.500000       19.047619           69.322967           0.225234       0.274766       
1         0.225234       -2.002870           90.276240           0.247420       0.022186       
2         0.247420       -0.037910           86.928274           0.247856       0.000436       
3         0.247856       -0.000013           86.866560           0.247856       0.000000       


0.24785591766721887

In [37]:
# GT3 Problem 1 Part A Solution
phi = Newton_Raphson(phi_0, num_components, comp_x, K, show_table=False)
V_F = phi * np.sum(comp_x)
L_F= (1-phi)*np.sum(comp_x)
print(f"Vapor flow rate: {V_F}")
print(f"Liquid flow rate: {L_F}")

Vapor flow rate: 24.785591766721886
Liquid flow rate: 75.21440823327812


In [45]:
#GT3 Problem 1 Part B Solution
# Calculate x and y for each component
x = comp_x / (1 + phi * (K - 1))/100
y = K * x

print(f"Calculated x values: {x}") #In order of component
print(f"Calculated y values: {y}") #In order of component

Calculated x values: [0.19413919 0.80586081]
Calculated y values: [0.62124542 0.37875458]


In [52]:
import numpy as np

def straight_line_error_method(T1, error1, T2, error2):
    """
    Calculate a new temperature using the straight-line error method.

    Parameters:
    T1 : float
        Temperature from the first trial.
    error1 : float
        Error from the first trial.
    T2 : float
        Temperature from the second trial.
    error2 : float
        Error from the second trial.

    Returns:
    new_T : float
        New temperature calculated by linear interpolation.
    """
    # Calculate the slope of the error function
    slope = (error2 - error1) / (T2 - T1)

    # Use linear interpolation to calculate the new temperature
    new_T = T2 - error2 / slope
    return new_T

def bubble_point_temperature(pressure, comp_x, num_components):
    """
    Calculate the bubble point temperature using user input and the straight-line error method.

    Parameters:
    pressure : float
        Pressure at which the bubble point temperature is to be determined (in mmHg).
    comp_x : list or numpy array
        Mole fractions of the components in the mixture.
    num_components : int
        Number of components in the mixture.

    Returns:
    None
    """
    # Prepare to store results for the table
    results = []

    # First trial: Initial guess
    T_guess_1 = float(input("Enter the initial guess temperature (in °C) for trial 1: "))
    K_values_1 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_1}°C: ")) for i in range(num_components)]

    # Calculate the error for the first trial
    error_1 = 1 - np.sum([K_values_1[i] * comp_x[i] for i in range(num_components)])
    results.append((T_guess_1, K_values_1, error_1))

    print(f"Error for Trial 1 (T={T_guess_1}°F): {error_1:.5f}")

    # Second trial: Adjust temperature and K values
    T_guess_2 = float(input("Enter the next temperature guess (in °F) for trial 2: "))
    K_values_2 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_2}°F: ")) for i in range(num_components)]

    # Calculate the error for the second trial
    error_2 = 1 - np.sum([K_values_2[i] * comp_x[i] for i in range(num_components)])
    results.append((T_guess_2, K_values_2, error_2))

    print(f"Error for Trial 2 (T={T_guess_2}°F): {error_2:.5f}")

    # Calculate the new temperature using the straight-line error method
    new_T = straight_line_error_method(T_guess_1, error_1, T_guess_2, error_2)

    # Calculate the error for the new temperature
    K_values_new = [float(input(f"Enter the K value for component {i+1} at T={new_T}°F: ")) for i in range(num_components)]
    error_new = 1 - np.sum([K_values_new[i] * comp_x[i] for i in range(num_components)])
    results.append((new_T, K_values_new, error_new))

    print(f"New temperature after straight-line method: T={new_T:.2f}°F with error {error_new:.5f}")

    # Print the results in a table format
    print(f"\n{'Trial Temp (°F)':<20}{'K values':<40}{'Error':<20}")
    print("="*80)
    for T_guess, K_values, error in results:
        print(f"{T_guess:<20}{' '.join([f'{k:.2f}' for k in K_values]):<40}{error:<20.5f}")




In [53]:
#GT3 Problem 1 Part C Solution
pressure = 100  # Atmospheric pressure in psia
bubble_point_temperature(pressure, comp_x, num_components)

Enter the initial guess temperature (in °C) for trial 1: 140
Enter the K value for component 1 at T=140.0°C: 2.7
Enter the K value for component 2 at T=140.0°C: 0.351
Error for Trial 1 (T=140.0°C): -0.05570
Enter the next temperature guess (in °C) for trial 2: 130
Enter the K value for component 1 at T=130.0°C: 2.4
Enter the K value for component 2 at T=130.0°C: 0.3
Error for Trial 2 (T=130.0°C): 0.07000
Enter the K value for component 1 at T=135.56881463802705°C: 2.6
Enter the K value for component 2 at T=135.56881463802705°C: 0.34
New temperature after straight-line method: T=135.57°C with error -0.01800

Trial Temp (°C)     K values                                Error               
140.0               2.70 0.35                               -0.05570            
130.0               2.40 0.30                               0.07000             
135.56881463802705  2.60 0.34                               -0.01800            


In [56]:
def dew_point_temperature(pressure, comp_x, num_components):
    """
    Calculate the dew point temperature using user input and the straight-line error method.

    Parameters:
    pressure : float
        Pressure at which the dew point temperature is to be determined (in mmHg).
    comp_x : list or numpy array
        Mole fractions of the components in the mixture (liquid phase).
    num_components : int
        Number of components in the mixture.

    Returns:
    None
    """
    # Prepare to store results for the table
    results = []

    # First trial: Initial guess
    T_guess_1 = float(input("Enter the initial guess temperature (in °F) for trial 1: "))
    K_values_1 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_1}°F: ")) for i in range(num_components)]

    # Calculate the error for the first trial
    error_1 = 1 - np.sum([comp_x[i] / K_values_1[i] for i in range(num_components)])
    results.append((T_guess_1, K_values_1, error_1))

    print(f"Error for Trial 1 (T={T_guess_1}°F): {error_1:.5f}")

    # Second trial: Adjust temperature and K values
    T_guess_2 = float(input("Enter the next temperature guess (in °F) for trial 2: "))
    K_values_2 = [float(input(f"Enter the K value for component {i+1} at T={T_guess_2}°F: ")) for i in range(num_components)]

    # Calculate the error for the second trial
    error_2 = 1 - np.sum([comp_x[i] / K_values_2[i] for i in range(num_components)])
    results.append((T_guess_2, K_values_2, error_2))

    print(f"Error for Trial 2 (T={T_guess_2}°F): {error_2:.5f}")

    # Calculate the new temperature using the straight-line error method
    new_T = straight_line_error_method(T_guess_1, error_1, T_guess_2, error_2)

    # Calculate the error for the new temperature
    K_values_new = [float(input(f"Enter the K value for component {i+1} at T={new_T}°F: ")) for i in range(num_components)]
    error_new = 1 - np.sum([comp_x[i] / K_values_new[i] for i in range(num_components)])
    results.append((new_T, K_values_new, error_new))

    print(f"New temperature after straight-line method: T={new_T:.2f}°F with error {error_new:.5f}")

    # Print the results in a table format
    print(f"\n{'Trial Temp (°F)':<20}{'K values':<40}{'Error':<20}")
    print("="*80)
    for T_guess, K_values, error in results:
        print(f"{T_guess:<20}{' '.join([f'{k:.2f}' for k in K_values]):<40}{error:<20.5f}")



In [55]:
#Problem 1 Part D Solution
dew_point_temperature(pressure, comp_x, num_components)

Enter the initial guess temperature (in °C) for trial 1: 180
Enter the K value for component 1 at T=180.0°C: 3.65
Enter the K value for component 2 at T=180.0°C: 0.59
Error for Trial 1 (T=180.0°C): -0.26863
Enter the next temperature guess (in °C) for trial 2: 220
Enter the K value for component 1 at T=220.0°C: 4.8
Enter the K value for component 2 at T=220.0°C: 0.92
Error for Trial 2 (T=220.0°C): 0.17663
Enter the K value for component 1 at T=204.1324810728167°C: 4.35
Enter the K value for component 2 at T=204.1324810728167°C: 0.79
New temperature after straight-line method: T=204.13°C with error 0.04496

Trial Temp (°C)     K values                                Error               
180.0               3.65 0.59                               -0.26863            
220.0               4.80 0.92                               0.17663             
204.1324810728167   4.35 0.79                               0.04496             


In [60]:
#Problem 2
phi_0 = float(input("Input an initial guess for phi: "))
num_components = int(input("Input the number of streams feeding the flash drum: "))
comp_x = np.array(list(map(float, input("Input a list of molar flow rates for each stream (comma-separated): ").split(','))))
K = np.array(list(map(float, input("Input the K (equilibrium constant) values for each stream in order (comma-separated): ").split(','))))

Input an initial guess for phi: 0.4
Input the number of streams feeding the flash drum: 4
Input a list of molar flow rates for each stream (comma-separated): 1777.99899, 3298.98374, 400.01759000000004, 119.99968
Input the K (equilibrium constant) values for each stream in order (comma-separated): 84.15, 11.24, 0.01233, 0.00454


In [59]:
H=0.31767*5597
M=0.58942*5597
B=0.07147*5597
T=0.02144*5597
print(H, M, B, T)

1777.99899 3298.98374 400.01759000000004 119.99968


In [67]:
#GT3 Problem 2 Part A & B Work
phi= Newton_Raphson(phi_0, num_components, comp_x, K)
V_F = phi * np.sum(comp_x)
L_F= (1-phi)*np.sum(comp_x)
print(f"Vapor flow rate: {V_F}")
print(f"Liquid flow rate: {L_F}")

Iteration Phi            f(Phi)              f(Phi)              Phi_next       Error          
-----------------------------------------------------------------------------------------------
0         0.400000       -10092.697733       25188.415517        0.800688       0.400688       
1         0.800688       -3382.726585        18584.349178        0.982708       0.182020       
2         0.982708       14084.743043        707102.333689       0.962789       0.019919       
3         0.962789       5987.409227         235557.813613       0.937371       0.025418       
4         0.937371       2051.456658         102534.608888       0.917364       0.020007       
5         0.917364       418.340887          65254.519665        0.910953       0.006411       
6         0.910953       24.604219           57829.312217        0.910527       0.000425       
7         0.910527       0.094267            57387.143444        0.910526       0.000002       
8         0.910526       0.000001       

In [68]:
#GT3 Problem 2 Part C Solution
# Calculate x and y for each component
x = comp_x / (1 + phi * (K - 1))/100
y = K * x

print(f"Calculated x values: {x}") #In order of component
print(f"Calculated y values: {y}") #In order of component

Calculated x values: [ 0.23178123  3.19551786 39.72331338 12.81938753]
Calculated y values: [19.50439083 35.9176207   0.48978845  0.05820002]


In [69]:
-30/-0.55


54.54545454545454

In [71]:
0.8/(0.36+1)

0.5882352941176472